In [7]:
# Installation and Imports
!pip -q install langchain langchain-community langchain-text-splitters
!pip -q install pypdf pymupdf sentence-transformers
!pip -q install numpy pandas tqdm faiss-cpu
!pip -q install instructor pydantic transformers accelerate torch

# Imports
from pathlib import Path
import json, re, pickle
from tqdm import tqdm
from typing import List, Dict, Any

# LangChain imports
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# LLM imports
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from instructor import patch

# Pydantic for validation
from pydantic import BaseModel, Field, field_validator

# Optional: keep CPU thread usage sane in notebooks
if not torch.cuda.is_available():
    torch.set_num_threads(2)


In [8]:
# Configuration
BOOK_PDF_PATH = Path("../Sources/book.pdf")
SECTIONS_JSON_PATH = Path("./working/page_to_section.json")

# Verify files exist
assert BOOK_PDF_PATH.exists(), f"PDF not found: {BOOK_PDF_PATH}"
assert SECTIONS_JSON_PATH.exists(), f"Sections JSON not found: {SECTIONS_JSON_PATH}"

# Load sections mapping
print("Loading sections mapping...")
with open(SECTIONS_JSON_PATH, "r", encoding="utf-8") as f:
    sections_map = json.load(f)

# Ensure all keys are strings
sections_map = {str(k): v for k, v in sections_map.items()}
print(f"Loaded {len(sections_map)} page-to-section mappings")
def clean_page_text(text: str) -> str:
    """Light cleaning for a single page string."""
    if not text: 
        return ""
    
    # Remove common boilerplate text
    text = re.sub(r'Access for free at openstax\.org\.*', '', text)
    text = re.sub(r'LINK TO LEARNING.*?(?=\n|$)', '', text)  # Remove LINK TO LEARNING sections
    text = re.sub(r'Watch a brief video.*?(?=\n|$)', '', text)  # Remove video references
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)  # Remove URLs
    
    # Basic text normalization
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"-\n(\w)", r"\1", text)       # de-hyphenate linebreaks
    text = re.sub(r"[ \t]+\n", "\n", text)       # strip trailing spaces before newline
    text = re.sub(r"[ \t]{2,}", " ", text)       # collapse multi-spaces
    
    # Remove multiple consecutive newlines
    text = re.sub(r'\n\s*\n', '\n', text)
    
    return text.strip()

def split_into_word_windows(text: str, size: int, overlap: int) -> List[str]:
    """Split ONE page into overlapping word windows. Returns list of strings."""
    words = text.replace("\n", " ").split()
    if not words:
        return []
    
    chunks = []
    start_idx = 0
    
    while start_idx < len(words):
        # Take size words for this chunk
        chunk = words[start_idx:start_idx + size]
        if chunk:
            chunks.append(" ".join(chunk))
        
        # Move start_idx forward by (size - overlap) to create overlap
        start_idx += (size - overlap)
        
        # If we can't make a full chunk anymore, break
        if start_idx + size > len(words):
            # Add final chunk if there are remaining words
            remaining = words[start_idx:]
            if remaining:
                chunks.append(" ".join(remaining))
            break
            
    return chunks

def make_chunk_id(physical_page: int, idx: int) -> str:
    """Make a stable chunk id like 'p19_w003' that preserves page number."""
    return f"p{physical_page}_w{idx:03d}"

def process_pdf(pdf_path: str, 
               sections_map: dict,
               start_page: int = 19,    # Skip front matter
               end_page: int = 638,     # Stop before references
               logical_offset: int = 12,
               chunk_size: int = 220,    
               chunk_overlap: int = 40    
               ) -> List[Document]:
    """Process PDF while maintaining page boundaries and creating overlapping chunks."""
    
    loader = PyPDFLoader(str(pdf_path))
    docs_all = loader.load()
    
    # Filter pages between start_page and end_page
    page_docs = [d for d in docs_all if start_page <= int(d.metadata.get("page", 0)) + 1 <= end_page]
    chunks = []
    
    for d in tqdm(page_docs, desc="Processing pages"):
        physical_page = int(d.metadata.get("page", 0)) + 1
        logical_page = physical_page - logical_offset
        section = sections_map.get(str(logical_page), "")
        
        page_text = clean_page_text(d.page_content)
        windows = split_into_word_windows(
            text=page_text,
            size=chunk_size,
            overlap=chunk_overlap
        )
        
        for i, window in enumerate(windows):
            metadata = {
                "chunk_id": make_chunk_id(physical_page, i),
                "physical_page": physical_page,
                "logical_page": logical_page,
                "section": section,
                "source": str(pdf_path),
                "chunk_size_words": chunk_size,
                "chunk_overlap_words": chunk_overlap,
            }
            chunks.append(Document(page_content=window, metadata=metadata))
            
    return chunks

# Usage
chunks = process_pdf(
    pdf_path=BOOK_PDF_PATH,
    sections_map=sections_map,
    start_page=19,   # Skip front matter
    end_page=638    # Stop before references
)

print(f"Total chunks created: {len(chunks)}")
for c in chunks[:5]:
    print("-" * 60)
    print(f"id     : {c.metadata['chunk_id']}")
    print(f"page   : phys {c.metadata['physical_page']} | logical {c.metadata['logical_page']}")
    print(f"section: {c.metadata['section'] or '(none)'}")
    print(f"text   : {c.page_content}...")

Loading sections mapping...
Loaded 494 page-to-section mappings


Processing pages: 100%|██████████| 620/620 [00:00<00:00, 7450.22it/s]

Total chunks created: 1780
------------------------------------------------------------
id     : p19_w000
page   : phys 19 | logical 7
section: (none)
text   : FIGURE 1.1 Psychology is the scientific study of mind and behavior. (credit "background": modification of work by Nattachai Noogure; credit "top left": modification of work by Peter Shanks; credit "top middle": modification of work by "devinf"/Flickr; credit "top right": modification of work by Alejandra Quintero Sinisterra; credit "bottom left": modification of work by Gabriel Rocha; credit "bottom middle-left": modification of work by Caleb Roenigk; credit "bottom middle-right": modification of work by Staffan Scherz; credit "bottom right": modification of work by Czech Provincial Reconstruction Team) INTRODUCTION CHAPTER OUTLINE 1.1 What Is Psychology? 1.2 History of Psychology 1.3 Contemporary Psychology 1.4 Careers in Psychology Clive Wearing is an accomplished musician who lost his ability to form new memories when he beca

In [9]:
# STEP 3: EMBEDDINGS AND VECTOR STORE
"""
In this step, we create a semantic search index for our text chunks:

1. Create embeddings using HuggingFace's sentence-transformers
   - Using all-MiniLM-L6-v2: Efficient model that works well on CPU
   - Normalized embeddings for better cosine similarity

2. Build FAISS index for fast similarity search
   - FAISS is efficient for large-scale similarity search
   - Works well on CPU for our dataset size

3. Save index for reuse
   - Persists both the FAISS index and metadata
   - Allows quick reloading without recomputing embeddings
"""

# Configuration
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # Efficient CPU model
INDEX_SAVE_PATH = Path("./artifacts/faiss_index")                # Where to save the index
INDEX_SAVE_PATH.mkdir(parents=True, exist_ok=True)

# Initialize embedding model
print("Loading embedding model...")
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={"device": "cpu"},           # Force CPU - works well for this model
    encode_kwargs={"normalize_embeddings": True}  # Better for cosine similarity
)

# Create FAISS index from our chunks
print("\nBuilding FAISS index from chunks...")
vectorstore = FAISS.from_documents(
    documents=chunks,           # Our preprocessed text chunks
    embedding=embeddings       # The embedding model
)

# Save index for reuse
print(f"\nSaving index to {INDEX_SAVE_PATH}...")
vectorstore.save_local(str(INDEX_SAVE_PATH))

# Quick verification of saved index
print("\nVerifying saved index...")
reloaded_vectorstore = FAISS.load_local(
    folder_path=str(INDEX_SAVE_PATH),
    embeddings=embeddings,
    allow_dangerous_deserialization=True  # Required in notebooks
)

# Test a simple query to verify everything works
test_query = "What is psychology?"
test_results = reloaded_vectorstore.similarity_search(
    query=test_query,
    k=2  # Get top 2 results
)

print("\nTest Results:")
print(f"Query: {test_query}")
for i, doc in enumerate(test_results, 1):
    print(f"\nResult {i}:")
    print(f"Page: {doc.metadata['physical_page']} (logical: {doc.metadata['logical_page']})")
    print(f"Section: {doc.metadata['section']}")
    print(f"Text: {doc.page_content[:200]}...")

/var/folders/52/ns3sdnhj22n2q3hsq7mpbgv40000gn/T/ipykernel_7414/2111709763.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading embedding model...

Building FAISS index from chunks...

Saving index to artifacts/faiss_index...

Verifying saved index...

Test Results:
Query: What is psychology?

Result 1:
Page: 20 (logical: 8)
Section: introduction_to_psychology/what_is_psychology
Text: understanding of the mind is so limited, since thoughts, at least as we experience them, are neither matter nor energy. The scientific method is also a form of empiricism. An empirical method for acqu...

Result 2:
Page: 20 (logical: 8)
Section: introduction_to_psychology/what_is_psychology
Text: 2002). Nash was the subject of the 2001 movie A Beautiful Mind. Why did these people have these experiences? How does the human brain work? And what is the connection between the brain’s internal proc...


In [8]:
# STEP 4: MULTI-QUERY EXPANSION WITH FLAN-T5
"""
In this step, we implement query expansion using FLAN-T5-small:
1. Use FLAN-T5: Very lightweight model optimized for instructions
2. Works efficiently on CPU without quantization
3. Generate 4 semantically similar queries
4. Validate output format with Pydantic

Why FLAN-T5-small?
- Only 80M parameters
- Optimized for instruction following
- Works well on CPU without special optimizations
- Very low memory footprint (~300MB RAM)
- Fast inference time
"""

# Standard library imports
import json
import pickle
from pathlib import Path
from typing import List, Optional

# Third-party imports
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer
from pydantic import BaseModel, Field, field_validator

# Setup directories
artifacts_dir = Path("./artifacts")
artifacts_dir.mkdir(parents=True, exist_ok=True)

class QueryExpansion(BaseModel):
    """Validates the structure of expanded queries."""
    queries: List[str] = Field(
        ...,
        min_items=4,
        max_items=4,
        description="List of 4 semantically similar but differently phrased queries"
    )
    
    @field_validator("queries")
    def validate_queries(cls, queries: List[str]) -> List[str]:
        # Remove empty strings and whitespace
        queries = [q.strip() for q in queries if q.strip()]
        
        if len(queries) != 4:
            raise ValueError("Must have exactly 4 non-empty queries")
            
        # Check for duplicates
        if len(set(queries)) != len(queries):
            raise ValueError("All queries must be unique")
            
        return queries

# Prompt template for T5
INSTRUCTION_TEMPLATE = """Generate 4 different ways to ask this question while keeping the same meaning. Return a JSON object.

Question: {query}

Requirements:
- Keep the same meaning and scope
- Use different vocabulary and phrasings
- Make each version unique
- Return exactly 4 versions
- Focus on semantic variation

Return format:
{{
  "queries": [
    "variation 1",
    "variation 2",
    "variation 3",
    "variation 4"
  ]
}}

JSON output:"""

print("Loading FLAN-T5-small model...")
model_name = "google/flan-t5-small"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    device_map="cpu",
    torch_dtype=torch.float32
)

def generate_response(prompt: str, input_query: str, max_attempts: int = 3) -> str:
    """Generate response from T5 model with retries."""
    for attempt in range(max_attempts):
        try:
            # T5 specific tokenization
            inputs = tokenizer(
                prompt, 
                return_tensors="pt", 
                max_length=512, 
                truncation=True,
                padding=True
            )
            
            # T5 specific generation
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_length=150,  # Shorter for JSON output
                min_length=50,   # Ensure some content
                temperature=0.6 + (attempt * 0.1),  # Start cooler
                do_sample=True,
                top_p=0.9,
                num_return_sequences=1,
                repetition_penalty=1.2,  # Avoid repeating phrases
                length_penalty=1.0,  # Balanced length
                no_repeat_ngram_size=2  # Avoid repeating bigrams
            )
            
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Clean and validate JSON
            json_start = response.find('{')
            json_end = response.rfind('}') + 1
            if json_start >= 0 and json_end > json_start:
                json_str = response[json_start:json_end]
                # Parse and validate structure
                data = json.loads(json_str)
                if isinstance(data, dict) and "queries" in data and len(data["queries"]) == 4:
                    return json_str
                
        except Exception as e:
            if attempt == max_attempts - 1:
                print(f"Failed to generate valid response after {max_attempts} attempts: {str(e)}")
                break
            continue
    
    # Fallback to template-based variations
    return json.dumps({
        "queries": [
            input_query,
            f"Define {input_query.lower().replace('what is ', '')}",
            f"Explain the concept of {input_query.lower().replace('what is ', '')}",
            f"Describe {input_query.lower().replace('what is ', '')}"
        ]
    })

def expand_query(query: str) -> List[str]:
    """Generate 4 semantically similar variations of the input query."""
    
    # Format prompt with escaped braces for JSON template
    prompt = INSTRUCTION_TEMPLATE.format(query=query)
    
    try:
        # Generate response
        response_text = generate_response(prompt, query)  # Pass query to fallback
        
        # Extract JSON
        json_data = json.loads(response_text)
        if not json_data:
            raise ValueError("Could not extract JSON from response")
            
        # Validate with Pydantic
        validated = QueryExpansion(**json_data)
        return validated.queries
        
    except Exception as e:
        print(f"Error generating variations: {e}")
        # Fallback variations
        return [
            query,
            f"Define {query.lower().replace('what is ', '')}",
            f"Explain the concept of {query.lower().replace('what is ', '')}",
            f"Describe {query.lower().replace('what is ', '')}"
        ]

# Test the expansion
test_query = "What is psychology?"
print("\nOriginal query:", test_query)

expanded_queries = expand_query(test_query)
print("\nExpanded queries:")
for i, q in enumerate(expanded_queries, 1):
    print(f"{i}. {q}")

# Save expanded queries for next step
print("\nSaving expanded queries...")
with open("artifacts/expanded_queries.pkl", "wb") as f:
    pickle.dump({
        "original_query": test_query,
        "expanded_queries": expanded_queries
    }, f)

Loading FLAN-T5-small model...

Original query: What is psychology?

Expanded queries:
1. What is psychology?
2. Define psychology?
3. Explain the concept of psychology?
4. Describe psychology?

Saving expanded queries...


In [12]:
# STEP 5: COSINE SIMILARITY RERANKING
"""
In this step, we:
1. Get initial candidates (5 per query) using FAISS
2. Rerank using cosine similarity
3. Select top 3 chunks per query
4. Structure results with metadata

Why cosine similarity?
- Fast and efficient
- Works well with normalized embeddings
- No additional models needed
- Purely CPU-based computation
"""

# Standard library imports
from typing import List, Dict, Any, Tuple
import numpy as np
from dataclasses import dataclass

# Third-party imports
from tqdm.auto import tqdm
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Reload FAISS index
print("Reloading FAISS index...")
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

vectorstore = FAISS.load_local(
    folder_path="artifacts/faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

# Load saved queries
print("\nLoading expanded queries...")
with open("artifacts/expanded_queries.pkl", "rb") as f:
    query_data = pickle.load(f)
    original_query = query_data["original_query"]
    expanded_queries = query_data["expanded_queries"]

print(f"\nOriginal query: {original_query}")
print(f"Number of variations: {len(expanded_queries)}")

def get_initial_chunks(query: str, k: int = 5) -> List[Document]:
    """Get initial chunks using FAISS similarity search."""
    return vectorstore.similarity_search_with_score(
        query=query,
        k=k
    )

def process_query(query: str) -> Dict[str, Any]:
    """Process a single query and return top chunks with metadata."""
    # Get initial chunks with scores
    chunks_and_scores = get_initial_chunks(query, k=5)
    
    # Sort by score (cosine similarity)
    sorted_chunks = sorted(chunks_and_scores, key=lambda x: x[1], reverse=True)
    
    # Take top 3
    top_chunks = sorted_chunks[:3]
    
    # Format results
    return {
        "query": query,
        "chunks": [
            {
                "score": float(score),  # Already cosine similarity
                "content": doc.page_content,
                "metadata": doc.metadata
            }
            for doc, score in top_chunks
        ]
    }

# Process all queries
print("\nProcessing queries...")
all_results = {
    "original_query": original_query,
    "results": []
}

for query in tqdm(expanded_queries, desc="Processing queries"):
    result = process_query(query)
    all_results["results"].append(result)

# Save results
print("\nSaving results...")
with open("artifacts/ranked_results.pkl", "wb") as f:
    pickle.dump(all_results, f)

# Print sample results
print("\nSample results for first query:")
first_result = all_results["results"][0]
print(f"\nQuery: {first_result['query']}")
print(f"Top 3 chunks:")
for i, chunk in enumerate(first_result["chunks"], 1):
    print(f"\n{i}. Score: {chunk['score']:.3f}")
    print(f"   Page: {chunk['metadata']['physical_page']} (logical: {chunk['metadata']['logical_page']})")
    print(f"   Section: {chunk['metadata']['section']}")
    print(f"   Text: {chunk['content'][:200]}...")

Reloading FAISS index...

Loading expanded queries...

Original query: What is psychology?
Number of variations: 4

Processing queries...


Processing queries: 100%|██████████| 4/4 [00:00<00:00, 40.37it/s]


Saving results...

Sample results for first query:

Query: What is psychology?
Top 3 chunks:

1. Score: 0.736
   Page: 43 (logical: 31)
   Section: 
   Text: of the most influential schools of thought within psychology’s history was behaviorism. Behaviorism focused on making psychology an objective science by studying overt behavior and deemphasizing the i...

2. Score: 0.731
   Page: 42 (logical: 30)
   Section: 
   Text: Key Terms American Psychological Association (APA) professional organization representing psychologists in the United States behaviorism focus on observing and controlling behavior biopsychology study...

3. Score: 0.705
   Page: 43 (logical: 31)
   Section: 
   Text: made up of several major subdivisions with unique perspectives. Biological psychology involves the study of the biological bases of behavior. Sensation and perception refer to the area of psychology t...


In [8]:
# STEP 6: FINAL ANSWER GENERATION
import json
import pickle
import gc
from pathlib import Path
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Enable garbage collection
gc.enable()

# Load ranked results
print("Loading ranked results...")
with open("artifacts/ranked_results.pkl", "rb") as f:
    all_results = pickle.load(f)

original_query = all_results["original_query"]
query_results = all_results["results"]

def format_context_with_metadata(results):
    """Format retrieved chunks with full metadata."""
    formatted_results = []
    
    # Process each query variation
    for query_idx, query_result in enumerate(results):
        query_info = {
            "query": query_result["query"],
            "chunks": []
        }
        
        # Process each chunk
        for chunk in query_result["chunks"][:2]:  # Top 2 chunks per query
            chunk_info = {
                "content": chunk["content"].strip(),
                "metadata": {
                    "physical_page": chunk["metadata"]["physical_page"],
                    "logical_page": chunk["metadata"]["logical_page"],
                    "section": chunk["metadata"]["section"]
                },
                "score": chunk["score"]
            }
            query_info["chunks"].append(chunk_info)
        
        formatted_results.append(query_info)
    
    return formatted_results

def format_display_context(formatted_results):
    """Format context for the model prompt."""
    context_parts = []
    
    for query_result in formatted_results:
        for chunk in query_result["chunks"]:
            content = chunk["content"]
            page = chunk["metadata"]["physical_page"]
            section = chunk["metadata"]["section"]
            
            # Format with detailed citation
            context_parts.append(
                f"[Page {page} - {section}]:\n{content}"
            )
    
    return "\n\n".join(context_parts)

# Improved prompt template
ANSWER_TEMPLATE = """Answer the question using only the provided context. Include page citations [Page X] to support your answer.

Question: {question}

Context:
{context}

Instructions:
1. Answer directly and clearly
2. Use information only from the context
3. Include page citations [Page X]
4. Be concise but complete
5. Focus on explaining concepts, not just listing terms

Answer:"""

print("\nLoading T5-small model...")
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="cpu",
    low_cpu_mem_usage=True
)

def generate_answer(question: str, context: str) -> str:
    """Generate answer using T5-small with optimized parameters."""
    try:
        # Prepare input
        prompt = ANSWER_TEMPLATE.format(
            question=question,
            context=context
        )
        
        # Tokenize with length limits
        inputs = tokenizer(
            prompt,
            max_length=512,
            truncation=True,
            return_tensors="pt"
        )
        
        # Generate with improved parameters
        outputs = model.generate(
            inputs.input_ids,
            max_length=200,
            min_length=50,
            num_beams=3,
            temperature=0.7,
            no_repeat_ngram_size=3,
            early_stopping=True,
            repetition_penalty=1.2,
            length_penalty=1.0
        )
        
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Post-process to ensure citations are preserved
        if "[Page" not in answer:
            answer += "\n\nNote: This answer is based on information from the provided context."
        
        return answer
        
    except Exception as e:
        print(f"Error in generation: {e}")
        return f"Error generating response: {str(e)}"
    finally:
        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()

# Process results with metadata
print("\nProcessing results...")
formatted_results = format_context_with_metadata(query_results)
display_context = format_display_context(formatted_results)

# Generate final answer
print("\nGenerating final answer...")
final_answer = generate_answer(original_query, display_context)

# Save complete response with full metadata
print("\nSaving complete response...")
complete_response = {
    "question": original_query,
    "queries_and_results": formatted_results,
    "generated_answer": final_answer,
    "model_used": model_name,
    "generation_parameters": {
        "max_length": 200,
        "min_length": 50,
        "num_beams": 3,
        "temperature": 0.7,
        "no_repeat_ngram_size": 3,
        "early_stopping": True,
        "repetition_penalty": 1.2,
        "length_penalty": 1.0
    }
}

with open("artifacts/final_response.json", "w", encoding="utf-8") as f:
    json.dump(complete_response, f, indent=2, ensure_ascii=False)

# Display results
print("\nFinal Answer:")
print("-" * 80)
print(final_answer)
print("-" * 80)
print("\nFull results including metadata saved to artifacts/final_response.json")

# Cleanup
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

Loading ranked results...

Loading T5-small model...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Processing results...

Generating final answer...

Saving complete response...

Final Answer:
--------------------------------------------------------------------------------
Psychology is a diverse discipline that is made up of several major subdivisions with unique perspectives. Biological psychology involves the study of the biological bases of behavior. Sensation and perception refer to the area of psychology that is focused on how information from our sensory modalities is [Page 43 - ]:
--------------------------------------------------------------------------------

Full results including metadata saved to artifacts/final_response.json
